In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
#plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

### 데이터 준비

In [3]:
# 파일 불러오기
pit_all_data = pd.read_csv('투수_스탯_데이터_합본.csv')

sal_24 = pd.read_csv('2024_연봉_statiz.csv')
sal_25 = pd.read_csv('2025_연봉_statiz.csv')

rookie_24 = pd.read_csv('KBO_2023_신인_드래프트.csv') # 신인 선수 계약금
rookie_25 = pd.read_csv('KBO_2024_신인_드래프트.csv') 

foregin_24 = pd.read_csv('2024 선수단 등록 현황 자료.csv', encoding='euc-kr')# 외국인 선수
foregin_25 = pd.read_csv('2025 선수단 등록 현황 자료.csv', encoding='euc-kr')

In [4]:
rookie_25.rename(columns={'지명한 구단': '팀'}, inplace=True)

In [5]:
# 영어 이름 지우기 (한글로 작성) 
foregin_25['성명'] = foregin_25['성명'].str.replace(r'[\n\\(].*$', '', regex=True).str.strip()
foregin_24['성명'] = foregin_24['성명'].str.replace(r'[\n\\(].*$', '', regex=True).str.strip()

# 성 지우기 (연봉 데이터 파일과 선수이름 맞추는 과정) 
# 이름 문자열의 마지막 단어만 추출
foregin_25['성명'] = foregin_25['성명'].str.split().str[-1]
foregin_24['성명'] = foregin_24['성명'].str.split().str[-1]

# 구단 명 맞추기 
team_name_map = {
    '한화': '한화 이글스',
    '두산': '두산 베어스',
    'KIA': 'KIA 타이거즈',
    'kt': 'KT 위즈',
    'KT': 'KT 위즈',
    'LG': 'LG 트윈스',
    '롯데': '롯데 자이언츠',
    'NC': 'NC 다이노스',
    '삼성': '삼성 라이온즈',
    'SSG': 'SSG 랜더스',
    '키움': '키움 히어로즈',
}

rookie_25['팀'] = rookie_25['팀'].replace(team_name_map)
foregin_25['구단'] = foregin_25['구단'].replace(team_name_map)
foregin_24['구단'] = foregin_24['구단'].replace(team_name_map)

In [6]:
# 포지션이 '투수'인 선수만 필터링
rookie_24 = rookie_24[rookie_24['포지션'] == '투수']
rookie_25 = rookie_25[rookie_25['포지션'] == '투수']

# 연봉 데이터에 연도 추가 
sal_24['연도'] = 2024
sal_25['연도'] = 2025

# 칼럼 추가 (신인 구분) 
rookie_24['신인여부'] = True
rookie_24['연도'] = 2024

rookie_25['신인여부'] = True
rookie_25['연도'] = 2025

# 국적 칼럼의 데이터 변경
foregin_25['국적'] = "외국인"

In [7]:
# 이적 선수 데이터 처리 
foregin_24 = foregin_24.drop(index=13)
foregin_24.loc[14, '입단연도'] = '2024'

In [8]:
# 국적 칼럼의 데이터 변경
foregin_24['국적'] = "외국인"

In [9]:
pit_23 = pit_all_data[pit_all_data['연도'] == 2023]
pit_24 = pit_all_data[pit_all_data['연도'] == 2024]

In [10]:
# 연봉 단위 통일 (만원) 
rookie_24.rename(columns={'계약금': '계약금(만원)'}, inplace=True)
rookie_24['계약금(만원)'] = pd.to_numeric(rookie_24['계약금(만원)'], errors='coerce') / 10000

rookie_25.rename(columns={'계약금': '계약금(만원)'}, inplace=True)
rookie_25['계약금(만원)'] = pd.to_numeric(rookie_25['계약금(만원)'], errors='coerce') / 10000

In [11]:
# 변수 타입 변경 
rookie_24['계약금(만원)'] = rookie_24['계약금(만원)'].astype(int)
rookie_25['계약금(만원)'] = rookie_25['계약금(만원)'].astype(int)

In [12]:
# 연봉($)를 연봉(만원)으로 변환하기 
foregin_24['연봉(만원)'] = foregin_24['연봉($)'] * 1300 / 10000
foregin_25['연봉(만원)'] = foregin_25['연봉($)'] * 1300 / 10000

# 계약금($)를 계약금(만원)으로 변환하기 
foregin_24['계약금($)'] = pd.to_numeric(foregin_24['계약금($)'], errors='coerce')
foregin_24['계약금(만원)'] = foregin_24['계약금($)'] * 1300 / 10000

foregin_25['계약금($)'] = pd.to_numeric(foregin_25['계약금($)'], errors='coerce')
foregin_25['계약금(만원)'] = foregin_25['계약금($)'] * 1300 / 10000

In [13]:
# 변수 타입 변경 
foregin_24 = foregin_24.dropna(subset=['계약금(만원)'])
foregin_24['계약금(만원)'] = foregin_24['계약금(만원)'].astype(int)
foregin_24['연봉(만원)'] = foregin_24['연봉(만원)'].astype(int)

foregin_25 = foregin_25.dropna(subset=['계약금(만원)'])
foregin_25['계약금(만원)'] = foregin_25['계약금(만원)'].astype(int)
foregin_25['연봉(만원)'] = foregin_25['연봉(만원)'].astype(int)

In [14]:
foregin_24['입단연도'] = foregin_24['입단연도'].str.strip()
foregin_24['입단연도'] = foregin_24['입단연도'].astype(int)

In [15]:
foregin_24 = foregin_24[foregin_24['입단연도'] == 2024]
foregin_25 = foregin_25[foregin_25['입단연도'] == 2025]

In [16]:
# 외국인 선수 연봉 수정하기
foregin_24['연봉(만원)'] = np.where(
    foregin_24['입단연도'] == 2024,
    foregin_24['계약금(만원)'] + foregin_24['연봉(만원)'],
    foregin_24['연봉(만원)']
)

foregin_25['연봉(만원)'] = np.where(
    foregin_25['입단연도'] == 2025,
    foregin_25['계약금(만원)'] + foregin_25['연봉(만원)'],
    foregin_25['연봉(만원)']
)

foregin_24 = foregin_24.drop(columns=['계약금(만원)'])
foregin_25 = foregin_25.drop(columns=['계약금(만원)'])

In [17]:
# 칼럼 이름 통일 
foregin_24.rename(columns={'구단': '팀', '성명': '선수'}, inplace=True)
foregin_25.rename(columns={'구단': '팀', '성명': '선수'}, inplace=True)

In [18]:
# 필요한 칼럼만 가져오기 
foregin_24 = foregin_24[['선수', '연봉(만원)', '팀', '국적', '입단연도']]
foregin_25 = foregin_25[['선수', '연봉(만원)', '팀', '국적', '입단연도']]

sal_24.drop(['WAR', 'WAR당 연봉'], axis=1, inplace=True)
sal_25.drop(['WAR', 'WAR당 연봉'], axis=1, inplace=True)

In [19]:
# 선수 이름과 팀 이름 모두 일치하는 데이터가 있는지 확인
dup_list = sal_24[sal_24.duplicated(subset=['선수', '팀'], keep=False)][['선수', '팀']].drop_duplicates()
dup_list

,선수,팀
439,이승현,삼성 라이온즈
440,김태훈,삼성 라이온즈


In [20]:
# 신인선수 중복 확인 
result = rookie_24.query("선수 == '이승현' or 선수 == '김태훈'")
result

,팀,선수,계약금(만원),포지션,신인여부,연도


In [21]:
#display(pit_23.head(2))
#display(pit_24.head(2))

display(sal_24.head(2))
display(sal_25.head(2))

display(rookie_24.head(2))
display(rookie_25.head(2))

,선수,연봉(만원),팀,pid,연도
0,류현진,250000,한화 이글스,10590,2024
1,채은성,100000,한화 이글스,11215,2024


,선수,연봉(만원),팀,pid,연도
0,류현진,200000,한화 이글스,10590,2025
1,폰세,112000,한화 이글스,16313,2025


,팀,선수,계약금(만원),포지션,신인여부,연도
0,한화 이글스,김서현,50000,투수,True,2024
3,한화 이글스,김관우,9000,투수,True,2024


,팀,선수,포지션,계약금(만원),신인여부,연도
0,한화 이글스,황준서,투수,35000,True,2025
1,두산 베어스,김택연,투수,35000,True,2025


In [22]:
sal_24 = pd.merge(
    sal_24,
    rookie_24[['팀', '선수', '계약금(만원)', '신인여부']],
    on=['팀', '선수'],
    how='left' 
)

In [23]:
sal_25 = pd.merge(
    sal_25,
    rookie_25[['팀', '선수', '계약금(만원)', '신인여부']],
    on=['팀', '선수'],
    how='left' 
)

In [24]:
sal_24['계약금(만원)'] = sal_24['계약금(만원)'].fillna(0)
sal_24['계약금(만원)'] = sal_24['계약금(만원)'].astype(int)

sal_25['계약금(만원)'] = sal_25['계약금(만원)'].fillna(0)
sal_25['계약금(만원)'] = sal_25['계약금(만원)'].astype(int)

sal_24['신인여부'] = sal_24['신인여부'].fillna(False)
sal_25['신인여부'] = sal_25['신인여부'].fillna(False)

display(sal_24)
display(sal_25)

,선수,연봉(만원),팀,pid,연도,계약금(만원),신인여부
0,류현진,250000,한화 이글스,10590,2024,0,False
1,채은성,100000,한화 이글스,11215,2024,0,False
2,페냐,65000,한화 이글스,15146,2024,0,False
3,페라자,60000,한화 이글스,16121,2024,0,False
4,최재훈,60000,한화 이글스,10170,2024,0,False
...,...,...,...,...,...,...,...
850,박준형,3000,키움 히어로즈,14211,2024,0,False
851,박윤성,3000,키움 히어로즈,15475,2024,9000,True
852,서유신,3000,키움 히어로즈,15483,2024,0,False
853,이명기,3000,키움 히어로즈,14212,2024,0,False


,선수,연봉(만원),팀,pid,연도,계약금(만원),신인여부
0,류현진,200000,한화 이글스,10590,2025,0,False
1,폰세,112000,한화 이글스,16313,2025,0,False
2,플로리얼,98000,한화 이글스,16312,2025,0,False
3,엄상백,90000,한화 이글스,11318,2025,0,False
4,와이스,84000,한화 이글스,16153,2025,0,False
...,...,...,...,...,...,...,...
841,김주훈,3000,키움 히어로즈,16129,2025,7000,True
842,서유신,3000,키움 히어로즈,15483,2025,0,False
843,박성빈,3000,키움 히어로즈,15479,2025,0,False
844,이우석,1500,키움 히어로즈,11360,2025,0,False


In [25]:
display(foregin_24.head(2))
display(foregin_25.head(2))

,선수,연봉(만원),팀,국적,입단연도
1,엔스,117000,LG 트윈스,외국인,2024
6,더거,97500,SSG 랜더스,외국인,2024


,선수,연봉(만원),팀,국적,입단연도
1,올러,104000,KIA 타이거즈,외국인,2025
2,위즈덤,130000,KIA 타이거즈,외국인,2025


In [26]:
# 1. foreign_24에서 필요한 컬럼만 추출
temp = foregin_24[['선수', '팀', '연봉(만원)', '국적']].copy()
temp = temp.rename(columns={
    '연봉(만원)': '외국인연봉(만원)',  # 중복 방지용
    '국적': '국적'
})

# 2. sal_24에 병합 (left join: sal_24 기준)
sal_24 = pd.merge(
    sal_24, temp,
    on=['선수', '팀'],
    how='left'
)

# 3. 외국인 데이터가 있는 경우만 연봉(만원) 값 덮어쓰기
sal_24['연봉(만원)'] = sal_24['외국인연봉(만원)'].combine_first(sal_24['연봉(만원)'])

# 4. 불필요한 중간 컬럼 제거
sal_24.drop(columns=['외국인연봉(만원)'], inplace=True)

In [27]:
# 외국인 데이터 에서 필요한 컬럼만 추출
temp = foregin_25[['선수', '팀', '연봉(만원)', '국적']].copy()
temp = temp.rename(columns={
    '연봉(만원)': '외국인연봉(만원)',  # 중복 방지용
    '국적': '국적'
})

# 2. sal_25에 병합 
sal_25 = pd.merge(
    sal_25, temp,
    on=['선수', '팀'],
    how='left'
)

# 3. 외국인 데이터가 있는 경우만 연봉(만원) 값 덮어쓰기
sal_25['연봉(만원)'] = sal_25['외국인연봉(만원)'].combine_first(sal_25['연봉(만원)'])

# 4. 불필요한 중간 컬럼 제거
sal_25.drop(columns=['외국인연봉(만원)'], inplace=True)

In [28]:
display(sal_24)
display(sal_25)

,선수,연봉(만원),팀,pid,연도,계약금(만원),신인여부,국적
0,류현진,250000,한화 이글스,10590,2024,0,False,NaN
1,채은성,100000,한화 이글스,11215,2024,0,False,NaN
2,페냐,65000,한화 이글스,15146,2024,0,False,NaN
3,페라자,104000.0,한화 이글스,16121,2024,0,False,외국인
4,최재훈,60000,한화 이글스,10170,2024,0,False,NaN
...,...,...,...,...,...,...,...,...
850,박준형,3000,키움 히어로즈,14211,2024,0,False,NaN
851,박윤성,3000,키움 히어로즈,15475,2024,9000,True,NaN
852,서유신,3000,키움 히어로즈,15483,2024,0,False,NaN
853,이명기,3000,키움 히어로즈,14212,2024,0,False,NaN


,선수,연봉(만원),팀,pid,연도,계약금(만원),신인여부,국적
0,류현진,200000,한화 이글스,10590,2025,0,False,NaN
1,폰세,130000.0,한화 이글스,16313,2025,0,False,외국인
2,플로리얼,97500.0,한화 이글스,16312,2025,0,False,외국인
3,엄상백,90000,한화 이글스,11318,2025,0,False,NaN
4,와이스,84000,한화 이글스,16153,2025,0,False,NaN
...,...,...,...,...,...,...,...,...
841,김주훈,3000,키움 히어로즈,16129,2025,7000,True,NaN
842,서유신,3000,키움 히어로즈,15483,2025,0,False,NaN
843,박성빈,3000,키움 히어로즈,15479,2025,0,False,NaN
844,이우석,1500,키움 히어로즈,11360,2025,0,False,NaN


In [29]:
# 1. 숫자형 변환
sal_24['연봉(만원)'] = pd.to_numeric(sal_24['연봉(만원)'], errors='coerce').fillna(0)
sal_24['계약금(만원)'] = pd.to_numeric(sal_24['계약금(만원)'], errors='coerce').fillna(0)
sal_25['연봉(만원)'] = pd.to_numeric(sal_25['연봉(만원)'], errors='coerce').fillna(0)
sal_25['계약금(만원)'] = pd.to_numeric(sal_25['계약금(만원)'], errors='coerce').fillna(0)

# 2. 연봉 + 계약금
sal_24['연봉(만원)'] = sal_24['연봉(만원)'] + sal_24['계약금(만원)'] * (sal_24['계약금(만원)'] != 0)
sal_25['연봉(만원)'] = sal_25['연봉(만원)'] + sal_25['계약금(만원)'] * (sal_25['계약금(만원)'] != 0)

In [30]:
# 컬럼 제거
sal_24.drop(columns=['계약금(만원)'], inplace=True)
sal_25.drop(columns=['계약금(만원)'], inplace=True)

In [31]:
display(sal_24)
display(sal_25)

,선수,연봉(만원),팀,pid,연도,신인여부,국적
0,류현진,250000.0,한화 이글스,10590,2024,False,NaN
1,채은성,100000.0,한화 이글스,11215,2024,False,NaN
2,페냐,65000.0,한화 이글스,15146,2024,False,NaN
3,페라자,104000.0,한화 이글스,16121,2024,False,외국인
4,최재훈,60000.0,한화 이글스,10170,2024,False,NaN
...,...,...,...,...,...,...,...
850,박준형,3000.0,키움 히어로즈,14211,2024,False,NaN
851,박윤성,12000.0,키움 히어로즈,15475,2024,True,NaN
852,서유신,3000.0,키움 히어로즈,15483,2024,False,NaN
853,이명기,3000.0,키움 히어로즈,14212,2024,False,NaN


,선수,연봉(만원),팀,pid,연도,신인여부,국적
0,류현진,200000.0,한화 이글스,10590,2025,False,NaN
1,폰세,130000.0,한화 이글스,16313,2025,False,외국인
2,플로리얼,97500.0,한화 이글스,16312,2025,False,외국인
3,엄상백,90000.0,한화 이글스,11318,2025,False,NaN
4,와이스,84000.0,한화 이글스,16153,2025,False,NaN
...,...,...,...,...,...,...,...
841,김주훈,10000.0,키움 히어로즈,16129,2025,True,NaN
842,서유신,3000.0,키움 히어로즈,15483,2025,False,NaN
843,박성빈,3000.0,키움 히어로즈,15479,2025,False,NaN
844,이우석,1500.0,키움 히어로즈,11360,2025,False,NaN


In [32]:
sal_24.loc[683, '국적'] = '외국인'
sal_24.loc[684, '국적'] = '외국인'
sal_24.loc[689, '국적'] = '외국인'

sal_24.loc[774, '국적'] = '외국인'
sal_24.loc[778, '국적'] = '외국인'
sal_24.loc[779, '국적'] = '외국인'

sal_24.loc[515, '국적'] = '외국인'
sal_24.loc[520, '국적'] = '외국인'
sal_24.loc[522, '국적'] = '외국인'

sal_24.loc[2, '국적'] = '외국인'
sal_24.loc[7, '국적'] = '외국인'
sal_24.loc[8, '국적'] = '외국인'

sal_24.loc[81, '국적'] = '외국인'
sal_24.loc[82, '국적'] = '외국인'
sal_24.loc[86, '국적'] = '외국인'

sal_24.loc[336, '국적'] = '외국인'
sal_24.loc[337, '국적'] = '외국인'
sal_24.loc[338, '국적'] = '외국인'

sal_24.loc[164, '국적'] = '외국인'
sal_24.loc[166, '국적'] = '외국인'

sal_24.loc[248, '국적'] = '외국인'
sal_24.loc[252, '국적'] = '외국인'

sal_24.loc[428, '국적'] = '외국인'
sal_24.loc[436, '국적'] = '외국인'

sal_24.loc[617, '국적'] = '외국인'

In [33]:
foreign_count_by_team = sal_24[sal_24['국적'] == '외국인'].groupby('팀').size()
print(foreign_count_by_team)

팀
KIA 타이거즈    4
KT 위즈       3
LG 트윈스      4
NC 다이노스     4
SSG 랜더스     4
두산 베어스      4
롯데 자이언츠     3
삼성 라이온즈     4
키움 히어로즈     3
한화 이글스      4
dtype: int64


In [34]:
sal_25.loc[767, '국적'] = '외국인'
sal_25.loc[768, '국적'] = '외국인'
sal_25.loc[769, '국적'] = '외국인'
sal_25.loc[772, '국적'] = '외국인'
sal_25.loc[773, '국적'] = '외국인'

sal_25.loc[679, '국적'] = '외국인'
sal_25.loc[681, '국적'] = '외국인'

sal_25.loc[591, '국적'] = '외국인'
sal_25.loc[593, '국적'] = '외국인'
sal_25.loc[594, '국적'] = '외국인'

sal_25.loc[427, '국적'] = '외국인'
sal_25.loc[431, '국적'] = '외국인'
sal_25.loc[434, '국적'] = '외국인'

sal_25.loc[516, '국적'] = '외국인'
sal_25.loc[518, '국적'] = '외국인'
sal_25.loc[519, '국적'] = '외국인'

sal_25.loc[337, '국적'] = '외국인'
sal_25.loc[339, '국적'] = '외국인'
sal_25.loc[344, '국적'] = '외국인'

sal_25.loc[4, '국적'] = '외국인'

sal_25.loc[89, '국적'] = '외국인'
sal_25.loc[91, '국적'] = '외국인'

sal_25.loc[171, '국적'] = '외국인'
sal_25.loc[172, '국적'] = '외국인'

sal_25.loc[252, '국적'] = '외국인'
sal_25.loc[768, '국적'] = '외국인'

sal_25.loc[10, '국적'] = '외국인'
sal_25.loc[535, '국적'] = '외국인'

In [35]:
foreign_count_by_team = sal_25[sal_25['국적'] == '외국인'].groupby('팀').size()
print(foreign_count_by_team)

팀
KIA 타이거즈    3
KT 위즈       4
LG 트윈스      3
NC 다이노스     3
SSG 랜더스     4
두산 베어스      3
롯데 자이언츠     3
삼성 라이온즈     4
키움 히어로즈     5
한화 이글스      4
dtype: int64


In [36]:
sal_24['국적'] = sal_24['국적'].fillna('내국인')
sal_25['국적'] = sal_25['국적'].fillna('내국인')

In [37]:
sal_24['연봉(만원)'] = pd.to_numeric(sal_24['연봉(만원)'], errors='coerce').fillna(0).astype(int)
sal_25['연봉(만원)'] = pd.to_numeric(sal_25['연봉(만원)'], errors='coerce').fillna(0).astype(int)

In [38]:
display(sal_24)
display(sal_25)

,선수,연봉(만원),팀,pid,연도,신인여부,국적
0,류현진,250000,한화 이글스,10590,2024,False,내국인
1,채은성,100000,한화 이글스,11215,2024,False,내국인
2,페냐,65000,한화 이글스,15146,2024,False,외국인
3,페라자,104000,한화 이글스,16121,2024,False,외국인
4,최재훈,60000,한화 이글스,10170,2024,False,내국인
...,...,...,...,...,...,...,...
850,박준형,3000,키움 히어로즈,14211,2024,False,내국인
851,박윤성,12000,키움 히어로즈,15475,2024,True,내국인
852,서유신,3000,키움 히어로즈,15483,2024,False,내국인
853,이명기,3000,키움 히어로즈,14212,2024,False,내국인


,선수,연봉(만원),팀,pid,연도,신인여부,국적
0,류현진,200000,한화 이글스,10590,2025,False,내국인
1,폰세,130000,한화 이글스,16313,2025,False,외국인
2,플로리얼,97500,한화 이글스,16312,2025,False,외국인
3,엄상백,90000,한화 이글스,11318,2025,False,내국인
4,와이스,84000,한화 이글스,16153,2025,False,외국인
...,...,...,...,...,...,...,...
841,김주훈,10000,키움 히어로즈,16129,2025,True,내국인
842,서유신,3000,키움 히어로즈,15483,2025,False,내국인
843,박성빈,3000,키움 히어로즈,15479,2025,False,내국인
844,이우석,1500,키움 히어로즈,11360,2025,False,내국인


In [39]:
# 파일 저장 
sal_24.to_csv('투수_2024_연봉_데이터_파생변수.csv', index=False)
sal_25.to_csv('투수_2025_연봉_데이터_파생변수.csv', index=False)